<a href="https://colab.research.google.com/github/parth06/Fake-News-Challenge/blob/master/Neural%20Net/Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/parth06/Fake-News-Challenge.git

In [2]:
cd Fake-News-Challenge/

/content/Fake-News-Challenge


In [3]:
!git submodule init
!git submodule update

Submodule 'fnc-1' (https://github.com/FakeNewsChallenge/fnc-1) registered for path 'fnc-1'
Cloning into '/content/Fake-News-Challenge/fnc-1'...
Submodule path 'fnc-1': checked out '29d473af2d15278f0464d5e41e4cbe7eb58231f2'


In [0]:
import nltk

In [5]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
!mkdir data/glove
!mkdir data/glove_twitter

!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
!wget http://nlp.stanford.edu/data/glove.6B.zip

!unzip glove.twitter.27B.zip -d data/glove_twitter/
!unzip glove.6B.zip -d data/glove

--2019-07-26 11:17:52--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2019-07-26 11:17:52--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2019-07-26 11:17:53--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [0]:
!rm glove.twitter.27B.zip
!rm glove.6B.zip

In [8]:
import gensim
import numpy as np
import pandas as pd
import os
import re
import csv
import codecs
import sys
import pickle

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

from keras import callbacks
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional
from keras.layers.merge import concatenate,subtract
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

import matplotlib as mpl
%matplotlib inline
from matplotlib import pyplot as plt
from keras.utils import plot_model 
from IPython.display import Image
import pydot
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import tensorflow as tf


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from string import punctuation

from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from keras import initializers, regularizers, constraints

tf_gpu_options = tf.GPUOptions(allow_growth = True) # per_process_gpu_memory_fraction=0.12,
tf_session = tf.Session(config=tf.ConfigProto(gpu_options=tf_gpu_options))
tf.keras.backend.set_session(tf_session)

#biodirectional embedding


Using TensorFlow backend.


In [0]:
# Specify the folder locations
#W2V_DIR = './data/GoogleNews-vectors-negative300.bin' #
GloVe_DIR = './data/glove_twitter/glove.twitter.27B.50d.txt'
#the data directory
DATA_DIR = './data/old'
# These are some hyperparameters that can be tuned
MAX_SENT_LEN = 300 #75(0.68), 150, 300 700(90% but too time comsuming)
MAX_VOCAB_SIZE = 28000 #vocabulary
LSTM_DIM = 50#len(embd[0])
EMBEDDING_DIM = 50 #50 for GloVe 300 for w2v
BATCH_SIZE = 128
N_EPOCHS = 40 #40

In [0]:
# fix random seed for reproducibility
seed = 1
np.random.seed(seed)

In [0]:
# Read the text files of fnc data
bodies = pd.read_csv(DATA_DIR + '/body_id.csv')
train_df = pd.read_csv(DATA_DIR + '/train.csv')
validation_df = pd.read_csv(DATA_DIR + '/validation.csv')
test_df = pd.read_csv(DATA_DIR + '/test.csv')


In [0]:
train_df.replace('unrelated',1,True)
train_df.replace('agree',2,True)
train_df.replace('disagree',3,True)
train_df.replace('discuss',4,True)

validation_df.replace('unrelated',1,True)
validation_df.replace('agree',2,True)
validation_df.replace('disagree',3,True)
validation_df.replace('discuss',4,True)

In [13]:
len(bodies["articleBody"][0])

1902

In [0]:
#Unrelated:1 Related:0
def f(row):
    if row['Stance']==4:
      return 1
    else:
      return 0

train_df['Classify'] = train_df.apply(f, axis=1)
validation_df['Classify'] = validation_df.apply(f, axis=1)

In [0]:
combine_df_train = train_df.join(bodies.set_index('Body ID'), on='Body ID')
combine_df_validation = validation_df.join(bodies.set_index('Body ID'), on='Body ID')
combine_df_test = test_df.join(bodies.set_index('Body ID'), on='Body ID')

In [0]:
# Pre-processing involves removal of puctuations and converting text to lower case
word_seq_head_train = [text_to_word_sequence(head) for head in combine_df_train['Headline']]
word_seq_bodies_train = [text_to_word_sequence(body) for body in combine_df_train['articleBody']]
word_seq_head_validation = [text_to_word_sequence(head) for head in combine_df_validation['Headline']]
word_seq_bodies_validation = [text_to_word_sequence(body) for body in combine_df_validation['articleBody']]
word_seq_head_test = [text_to_word_sequence(head) for head in combine_df_test['Headline']]
word_seq_bodies_test = [text_to_word_sequence(body) for body in combine_df_test['articleBody']]

In [17]:
print('90th Percentile Sentence of headline:', np.percentile([len(seq) for seq in word_seq_head_train], 90))
print('90th Percentile Sentence of body:', np.percentile([len(seq) for seq in word_seq_bodies_train], 50))

90th Percentile Sentence of headline: 16.0
90th Percentile Sentence of body: 308.0


In [0]:
word_seq = []
for i in range(len(word_seq_head_train)):
    word_seq.append(word_seq_head_train[i])
for i in range(len(word_seq_bodies_train)):
    word_seq.append(word_seq_bodies_train[i])

# for i in range(len(word_seq_head_validation)):
#     word_seq.append(word_seq_head_validation[i])
# for i in range(len(word_seq_bodies_validation)):
#     word_seq.append(word_seq_bodies_validation[i])
    
# for i in range(len(word_seq_head_test)):
#     word_seq.append(word_seq_head_test[i])
# for i in range(len(word_seq_bodies_test)):
#     word_seq.append(word_seq_bodies_test[i])

In [20]:
filter_list = '!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
MAX_VOCAB_SIZE = 25000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters=filter_list)
tokenizer.fit_on_texts([seq for seq in word_seq])
#because it only includes unique words(tokens)

print("Number of words in vocabulary:", len(tokenizer.word_index))

Number of words in vocabulary: 25128


In [21]:
# Load the word2vec embeddings 
#embeddings = gensim.models.KeyedVectors.load_word2vec_format(GloVe_DIR, binary=True)

#GloVes Load
glove_input_file = GloVe_DIR
word2vec_output_file = 'glove.50d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)
embeddings = gensim.models.KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1, EMBEDDING_DIM)) # +1 is because the matrix indices start with 0

for word, i in tokenizer.word_index.items(): # i=0 is the embedding for the zero padding
  try:
      embeddings_vector = embeddings[word]
  except KeyError:
      embeddings_vector = None
  if embeddings_vector is not None:
      embeddings_matrix[i] = embeddings_vector

del embeddings

In [0]:
MAX_HEAD_LEN = 15
MAX_BODY_LEN = 300
# Shorten the sentence to a fixed length
# Convert the sequence of words to sequnce of indices
X_head_train = tokenizer.texts_to_sequences([' '.join(seq[:MAX_HEAD_LEN]) for seq in word_seq_head_train])
X_head_train = pad_sequences(X_head_train, maxlen=MAX_HEAD_LEN, padding='post', truncating='post')

X_body_train = tokenizer.texts_to_sequences([' '.join(seq[:MAX_BODY_LEN]) for seq in word_seq_bodies_train])
X_body_train = pad_sequences(X_body_train, maxlen=MAX_BODY_LEN, padding='post', truncating='post')

y_stance_train = combine_df_train['Stance']
y_classify_train = combine_df_train['Classify']


#Validation
X_head_val = tokenizer.texts_to_sequences([' '.join(seq[:MAX_HEAD_LEN]) for seq in word_seq_head_validation])
X_head_val = pad_sequences(X_head_val, maxlen=MAX_HEAD_LEN, padding='post', truncating='post')

X_body_val = tokenizer.texts_to_sequences([' '.join(seq[:MAX_BODY_LEN]) for seq in word_seq_bodies_validation])
X_body_val = pad_sequences(X_body_val, maxlen=MAX_BODY_LEN, padding='post', truncating='post')

y_stance_val = combine_df_validation['Stance']
y_classify_val = combine_df_validation['Classify']

#TEST
# Convert the sequence of words to sequnce of indices
X_head_test = tokenizer.texts_to_sequences([' '.join(seq[:MAX_HEAD_LEN]) for seq in word_seq_head_test])
X_head_test = pad_sequences(X_head_test, maxlen=MAX_BODY_LEN, padding='post', truncating='post')

X_body_test = tokenizer.texts_to_sequences([' '.join(seq[:MAX_BODY_LEN]) for seq in word_seq_bodies_test])
X_body_test = pad_sequences(X_body_test, maxlen=MAX_BODY_LEN, padding='post', truncating='post')

In [0]:
#Encode class values as integers
encoder_train = LabelEncoder()
encoder_train.fit(y_stance_train)
encoded_train = encoder_train.transform(y_stance_train)
encoded_val = encoder_train.transform(y_stance_val)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_ys_train = np_utils.to_categorical(encoded_train)
dummy_ys_val = np_utils.to_categorical(encoded_val)

#Classify encoding
encoder_train = LabelEncoder()
encoder_train.fit(y_classify_train)
encoded_train = encoder_train.transform(y_classify_train)
encoded_val = encoder_train.transform(y_classify_val)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_yc_train = np_utils.to_categorical(encoded_train)
dummy_yc_val = np_utils.to_categorical(encoded_val)

In [25]:
len(X_body_train[1])

300

In [0]:
from keras.layers import add

In [27]:
# Build a sequential model by stacking neural net units 
left = Sequential()
left.add(Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embeddings_matrix], trainable=False, name='head_word_embedding_layer', #False
                          mask_zero=True))
left.add(Bidirectional(LSTM(LSTM_DIM, return_sequences=False, name='head_Bidrectional_lstm_layer1')))

right = Sequential()
right.add(Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embeddings_matrix], trainable=False, name='body_word_embedding_layer', #False
                          mask_zero=True))
right.add(Bidirectional(LSTM(LSTM_DIM, return_sequences=False, name='body_Bidrectional_lstm_layer2')))


concatenated = concatenate([left.output,right.output])
layer_1 = Dense(100, activation='relu', name='layer_1') (concatenated)
dropout_1 = Dropout(rate=0.5, name='dropout_1')(layer_1)
layer_2 =Dense(50,  activation='relu', name='layer_2') (dropout_1)
dropout_2 = Dropout(rate=0.5, name='dropout_2')(layer_2)
# layer_3 =Dense(600,  activation='relu', name='layer_3') (dropout_2)
# dropout_3 = Dropout(rate=0.5, name='dropout_3')(layer_3)
# out1 =  Dense(2, activation='softmax', name='output_layer1')(dropout_3)
out2 = Dense(4, activation='softmax', name='output_layer2')(dropout_2)

model = Model([left.input, right.input], out2)

model.summary()

W0726 11:28:25.067631 140354626779008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 11:28:25.071304 140354626779008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 11:28:25.084535 140354626779008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 11:28:25.098416 140354626779008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0726 11:28:25.099775 1403546267

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
head_word_embedding_layer_input (None, None)         0                                            
__________________________________________________________________________________________________
body_word_embedding_layer_input (None, None)         0                                            
__________________________________________________________________________________________________
head_word_embedding_layer (Embe (None, None, 50)     1256450     head_word_embedding_layer_input[0
__________________________________________________________________________________________________
body_word_embedding_layer (Embe (None, None, 50)     1256450     body_word_embedding_layer_input[0
__________________________________________________________________________________________________
bidirectio

In [0]:
from keras.utils import plot_model
plot_model(model, show_shapes=True)

In [0]:
!mkdir results

In [0]:
# filepath="./result/lstmBidirectional_lstm_50token_lr0.001_trainable_{epoch:02d}_{val_acc:.4f}.h5"
# checkpoint = callbacks.ModelCheckpoint(filepath, 
#                                        monitor='val_acc', 
#                                        verbose=0, 
#                                        save_best_only=True)
es = callbacks.EarlyStopping(monitor='val_acc', mode='max',patience=2)
callbacks_list1 = [es]

In [31]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

W0726 11:28:30.678092 140354626779008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
history = model.fit([X_head_train,X_body_train], y = dummy_ys_train,
          batch_size=BATCH_SIZE,
          epochs=N_EPOCHS,
                    verbose=1,
          validation_data=([X_head_val,X_body_val], dummy_ys_val),callbacks = callbacks_list1)



Train on 40350 samples, validate on 9622 samples
Epoch 1/40
40350/40350 [==============================] - 574s 14ms/step - loss: 0.8193 - acc: 0.7226 - val_loss: 0.6861 - val_acc: 0.7333
Epoch 2/40
40350/40350 [==============================] - 579s 14ms/step - loss: 0.6127 - acc: 0.7659 - val_loss: 0.6258 - val_acc: 0.7552
Epoch 3/40
40350/40350 [==============================] - 694s 17ms/step - loss: 0.4945 - acc: 0.8073 - val_loss: 0.5937 - val_acc: 0.7571
Epoch 4/40
40350/40350 [==============================] - 551s 14ms/step - loss: 0.4071 - acc: 0.8445 - val_loss: 0.5452 - val_acc: 0.7823
Epoch 5/40
40350/40350 [==============================] - 552s 14ms/step - loss: 0.3447 - acc: 0.8687 - val_loss: 0.5359 - val_acc: 0.8066
Epoch 6/40
40350/40350 [==============================] - 550s 14ms/step - loss: 0.2901 - acc: 0.8920 - val_loss: 0.5514 - val_acc: 0.8001
Epoch 7/40
 7552/40350 [====>.........................] - ETA: 6:57 - loss: 0.2640 - acc: 0.9020

In [0]:
model.evaluate([X_head_train,X_body_train],y=dummy_ys_train)

In [0]:
yt = model.predict([X_head_test,X_body_test])


In [0]:
yt = np.argmax(yt,axis=1)
ans = pd.DataFrame(yt)
ans.head()


In [0]:
ans.replace(0,'unrelated',True)
ans.replace(1,'agree',True)
ans.replace(2,'disagree',True)
ans.replace(3,'discuss',True)

In [0]:
import sys
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from feature_engineering import refuting_features, polarity_features, hand_features, gen_or_load_feats
from feature_engineering import word_overlap_features
from utils.dataset import DataSet
from utils.generate_test_splits import kfold_split, get_stances_for_folds
from utils.score import report_score, LABELS, score_submission

from utils.system import parse_params, check_version
def generate_features(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    return y
  

competition_dataset = DataSet("competition_test")
y_competition = generate_features(competition_dataset.stances, competition_dataset, "competition")

Xs = dict()
ys = dict()

# Load/Precompute all features now


In [0]:
#Run on competition dataset
actual = [LABELS[int(a)] for a in y_competition]

print("Scores on the test set")
report_score(actual,ans[0])


In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy(Truncation: 150 Epoch: 10)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()